<a href="https://colab.research.google.com/github/MengOonLee/Web_scraping/blob/master/Mydin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mydin

In [1]:
%%bash

pip install --no-cache-dir -qU pip wheel
pip install --no-cache-dir -qU scrapy
pip check

No broken requirements found.


In [ ]:
from scrapy import Selector
import requests

url = "https://mydinexpress.my/hypermart/product-category/grocery/"
html = requests.get(url).content

sel = Selector(text=html)
products = sel.xpath('//div[contains(@class, "product-grid-item")]')
for p in products:
    title = p.css('h3.product-title::text').extract_first()
    price = ''.join(p.css('span.price ::text').extract())
    print(title, ',', price)
    
next_page = sel.xpath('//div[@class="products-footer"]/a/@href').extract()
print(next_page)

In [3]:
%%bash
cd /home/meng/work/Airflow/dags/Scrapy
scrapy startproject Mydin

New Scrapy project 'Mydin', using template directory '/home/meng/venv/lib/python3.8/site-packages/scrapy/templates/project', created in:
    /home/meng/work/Airflow/dags/Scrapy/Mydin

You can start your first spider with:
    cd Mydin
    scrapy genspider example example.com


In [4]:
%%writefile /home/meng/work/Airflow/dags/Scrapy/Mydin/Mydin/spiders/grocery_spider.py
import scrapy
import datetime

class GrocerySpider(scrapy.Spider):
    name = 'grocery'
    start_urls = ["https://mydinexpress.my/hypermart/product-category/grocery/"]
    
    def parse(self, response):
        for product in response.xpath('//div[contains(@class, "product-grid-item")]'):
            yield {
                'name': product.css('h3.product-title::text').get(),
                'price': ''.join(product.css('span.price ::text').getall()),
                'datetime': datetime.datetime.now()
            }
            
        for next_page in response.xpath('//div[@class="products-footer"]/a/@href'):
            yield response.follow(next_page, self.parse)

Writing /home/meng/work/Airflow/dags/Scrapy/Mydin/Mydin/spiders/grocery_spider.py


In [13]:
import os
os.umask(0)
os.makedirs("/home/meng/work/Airflow/dags/Scrapy/Mydin/data", 
    exist_ok=True, mode=0o777)

In [55]:
%%writefile /home/meng/work/Airflow/dags/Scrapy/Mydin/dag.py
import os
import datetime
from textwrap import dedent
from airflow import DAG
from airflow.operators.bash import BashOperator

yesterday = datetime.datetime.combine(
    datetime.datetime.today() - datetime.timedelta(1),
    datetime.datetime.min.time())

with DAG(
    dag_id="Mydin_scrapy",
    default_args={
        "email": ["darklemonlee@yahoo.co.uk"],
        "email_on_failure": True
    },
    start_date=yesterday,
    schedule_interval="@daily",
    tags=["Test"]
) as dag:
    
    grocery = BashOperator(
        task_id="grocery",
        bash_command=dedent("""
            cd ${AIRFLOW_HOME}/dags/Scrapy/Mydin
            scrapy crawl grocery -O {{ params.filename }}_{{ ds_nodash }}.jl
        """),
        params={
            "filename": "./data/grocery"
        }
    )

Overwriting /home/meng/work/Airflow/dags/Scrapy/Mydin/dag.py
